Problematica 
Análisis de tendencias musicales a lo largo de los años.
Queremos analizar cómo ha evolucionado la popularidad de la música en diferentes géneros y cómo algunas características como la energía, bailabilidad, acústica y loudness han cambiado a lo largo de los años. A través de este análisis, se busca entender cómo los gustos musicales han cambiado y qué características de las canciones se correlacionan más con la popularidad a lo largo del tiempo.


In [1]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import json

COdigo principla: ElasticSearchProvider.
Este código sirve para conectarse a un servidor llamado Elasticsearch que está en tu computadora. Su función principal es leer un archivo de texto donde cada línea tiene datos en formato JSON (por ejemplo, información de canciones) y luego guardar todos esos datos en una especie de base de datos llamada "spotify-insights". Lo hace todo de forma rápida y automática. Además, cuando termina de cargar los datos, cierra la conexión con el servidor para no dejarla abierta. Es útil si tienes muchos datos y quieres guardarlos en un lugar donde luego puedas buscarlos fácilmente.

In [2]:
class ElasticSearchProvider:

    def __init__(self):
        self.host = "http://localhost:9200"
        #self.user = str(user)
        #self.password = str(password)
        self.index = "spotify-insights"
        self.index_type ="_doc"
        self.connection = Elasticsearch(self.host)

    def __enter__(self):
        try:
            self.connection=Elasticsearch(self.host)
            return self
        except Exception as e:
            return{
                "StatusCode":500,
                "body":json.dumps({
                    "Message":str(e)
                })
            }

    def __exit__(self, exception_type, exception_value, traceback):
        self.connection.transport.close()


Lista de consultas


1.- Top 10 canciones más populares por género:

In [3]:
def showMostPopularbyGenre(self):
        try:
            response=self.connection.search(
                index=self.index,
                body={
                    "size": 0,
                    "aggs": {
                        "top_genres": {
                        "terms": {
                            "field": "genre",
                            "size": 10
                        },
                        "aggs": {
                            "top_tracks": {
                            "top_hits": {
                                "size": 10,
                                "_source": ["track_name", "artist_name", "popularity"]
                                }
                            }
                        }
                    }
                }
            }
            )
            return response
            
        except Exception as e:
            print(f"Error: {str(e)}")

ElasticSearchProvider.showMostPopularbyGenre=showMostPopularbyGenre

In [6]:
provider = ElasticSearchProvider()
response = provider.showMostPopularbyGenre()
if response:
    print("Respuesta obtenida:")
    print(response)
else:
    print("No se obtuvo respuesta válida.")

Respuesta obtenida:
{'took': 20, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 10000, 'relation': 'gte'}, 'max_score': None, 'hits': []}, 'aggregations': {'top_genres': {'doc_count_error_upper_bound': 0, 'sum_other_doc_count': 951862, 'buckets': [{'key': 'black-metal', 'doc_count': 21852, 'top_tracks': {'hits': {'total': {'value': 21852, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'spotify-insights', '_id': 'ExEMYJYBMkMmnc_OPjLn', '_score': 1.0, '_source': {'artist_name': 'Kawir', 'track_name': 'Hail Bacchus', 'popularity': 12}}, {'_index': 'spotify-insights', '_id': 'FBEMYJYBMkMmnc_OPjLn', '_score': 1.0, '_source': {'artist_name': 'Kawir', 'track_name': 'Hymn to Apollo', 'popularity': 13}}, {'_index': 'spotify-insights', '_id': 'FREMYJYBMkMmnc_OPjLn', '_score': 1.0, '_source': {'artist_name': 'Hexis', 'track_name': 'Inferis', 'popularity': 10}}, {'_index': 'spotify-insights', '_id': 'FhEMYJYBMkMmnc_OPj

2.- Top 5 canciones más populares de un artista específico.

In [7]:
def showMostPopularbyArtist(self,name):
        try:
            response=self.connection.search(
                index=self.index,
                body={
                    "size": 5,
                    "query": {
                        "match": {
                        "artist_name": name 
                        }
                    },
                    "aggs": {
                        "top_tracks": {
                        "terms": {
                            "field": "track_name.keyword",  
                        },
                        "aggs": {
                            "popular_tracks": {
                            "top_hits": {
                                "size": 5,
                                "_source": ["track_name", "popularity"]
                            }
                            }
                        }
                        }
                    }
                }
            )
            return response
            
        except Exception as e:
            print(f"Error: {str(e)}")
            
ElasticSearchProvider.showMostPopularbyArtist=showMostPopularbyArtist

In [8]:
provider = ElasticSearchProvider()
name="Hexis"
response = provider.showMostPopularbyArtist(name)
if response:
    print("Respuesta obtenida:")
    print(response)
else:
    print("No se obtuvo respuesta válida.")

Respuesta obtenida:
{'took': 63, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 32, 'relation': 'eq'}, 'max_score': 10.482474, 'hits': [{'_index': 'spotify-insights', '_id': 'FREMYJYBMkMmnc_OPjLn', '_score': 10.482474, '_source': {'artist_name': 'Hexis', 'track_name': 'Inferis', 'popularity': 10, 'year': 2014, 'genre': 'black-metal', 'danceability': 0.127, 'energy': 0.247, 'loudness': -5.6, 'mode': 1, 'speechiness': 0.0365, 'acousticness': 4.65e-05, 'instrumentalness': 0.604, 'liveness': 0.298, 'valence': 0.0651, 'duration_ms': 536775, 'time_signature': 4}}, {'_index': 'spotify-insights', '_id': 'UREMYJYBMkMmnc_OPjPo', '_score': 10.482474, '_source': {'artist_name': 'Hexis', 'track_name': 'Evinco', 'popularity': 4, 'year': 2014, 'genre': 'black-metal', 'danceability': 0.122, 'energy': 0.032, 'loudness': -4.173, 'mode': 0, 'speechiness': 0.609, 'acousticness': 3.17e-06, 'instrumentalness': 0.878, 'liveness': 0.578, 'v

3.-Promedio de la popularidad por año.

In [16]:
def showMostPopularGenrebyYear(self):
        try:
            response=self.connection.search(
                index=self.index,
                body={
                    "size": 0,
                    "aggs": {
                        "avg_popularity_by_genre": {
                        "terms": {
                            "field": "genre",
                            "size": 20
                        },
                        "aggs": {
                            "avg_popularity": {
                            "avg": {
                                "field": "popularity"
                            }
                            }
                        }
                        }
                    }
                    }
            )
            return response
            
        except Exception as e:
            print(f"Error: {str(e)}")

ElasticSearchProvider.showMostPopularGenrebyYear=showMostPopularGenrebyYear

In [17]:
provider = ElasticSearchProvider()

response = provider.showMostPopularGenrebyYear()
if response:
    print("Respuesta obtenida:")
    print(response)
else:
    print("No se obtuvo respuesta válida.")

Respuesta obtenida:
{'took': 466, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 10000, 'relation': 'gte'}, 'max_score': None, 'hits': []}, 'aggregations': {'avg_popularity_by_genre': {'doc_count_error_upper_bound': 0, 'sum_other_doc_count': 764097, 'buckets': [{'key': 'black-metal', 'doc_count': 21852, 'avg_popularity': {'value': 11.089968881566904}}, {'key': 'gospel', 'doc_count': 21621, 'avg_popularity': {'value': 18.92040146154202}}, {'key': 'ambient', 'doc_count': 21389, 'avg_popularity': {'value': 20.008742811725654}}, {'key': 'acoustic', 'doc_count': 21097, 'avg_popularity': {'value': 17.825093615205954}}, {'key': 'alt-rock', 'doc_count': 20918, 'avg_popularity': {'value': 38.59680657806674}}, {'key': 'emo', 'doc_count': 20845, 'avg_popularity': {'value': 23.661981290477332}}, {'key': 'indian', 'doc_count': 20583, 'avg_popularity': {'value': 8.790458145071176}}, {'key': 'k-pop', 'doc_count': 20004, 'avg_popula

4.-Top 5 géneros más populares en un año específico.

In [18]:
def showMostPopularGenrebySpecificYear(self,year):
        try:
            response=self.connection.search(
                index=self.index,
                body={
                    "size": 0,
                    "query": {
                        "term": {
                        "year": year
                        }
                    },
                    "aggs": {
                        "top_genres": {
                        "terms": {
                            "field": "genre",
                            "size": 5
                        },
                        "aggs": {
                            "avg_popularity": {
                            "avg": {
                                "field": "popularity"
                            }
                            }
                        }
                        }
                    }
                    }
            )
            return response
            
        except Exception as e:
            print(f"Error: {str(e)}")
ElasticSearchProvider.showMostPopularGenrebySpecificYear=showMostPopularGenrebySpecificYear

In [19]:
provider = ElasticSearchProvider()
year=1577836800000
response = provider.showMostPopularGenrebySpecificYear(year)
if response:
    print("Respuesta obtenida:")
    print(response)
else:
    print("No se obtuvo respuesta válida.")

Respuesta obtenida:
{'took': 6, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 10000, 'relation': 'gte'}, 'max_score': None, 'hits': []}, 'aggregations': {'top_genres': {'doc_count_error_upper_bound': 0, 'sum_other_doc_count': 50302, 'buckets': [{'key': 'comedy', 'doc_count': 973, 'avg_popularity': {'value': 11.828365878725592}}, {'key': 'chill', 'doc_count': 965, 'avg_popularity': {'value': 45.63316062176166}}, {'key': 'acoustic', 'doc_count': 951, 'avg_popularity': {'value': 29.52155625657203}}, {'key': 'sleep', 'doc_count': 939, 'avg_popularity': {'value': 28.78168264110756}}, {'key': 'indian', 'doc_count': 936, 'avg_popularity': {'value': 21.661324786324787}}]}}}


5.-Top 10 canciones con la mayor energía.

In [20]:
def showSongsWhittheMostEnegy(self):
        try:
            response=self.connection.search(
                index=self.index,
                body={
                    "size": 0,
                    "aggs": {
                        "top_energy_songs": {
                        "top_hits": {
                            "size": 10,
                            "_source": ["track_name", "artist_name", "energy"],
                            "sort": [
                            {
                                "energy": {
                                "order": "desc"
                                }
                            }
                            ]
                        }
                        }
                    }
                    }
            )
            return response
            
        except Exception as e:
            print(f"Error: {str(e)}")
            
ElasticSearchProvider.showSongsWhittheMostEnegy=showSongsWhittheMostEnegy

In [21]:
provider = ElasticSearchProvider()
response = provider.showSongsWhittheMostEnegy()
if response:
    print("Respuesta obtenida:")
    print(response)
else:
    print("No se obtuvo respuesta válida.")

Respuesta obtenida:
{'took': 18, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 10000, 'relation': 'gte'}, 'max_score': None, 'hits': []}, 'aggregations': {'top_energy_songs': {'hits': {'total': {'value': 1159764, 'relation': 'eq'}, 'max_score': None, 'hits': [{'_index': 'spotify-insights', '_id': 'rRsOYJYBMkMmnc_OKuIO', '_score': None, '_source': {'artist_name': 'Cascada', 'track_name': '(I Need A) Miracle [S.A.D. Radio Mix]', 'energy': 1.0}, 'sort': [1.0]}, {'_index': 'spotify-insights', '_id': 'PxwOYJYBMkMmnc_OMR9Q', '_score': None, '_source': {'artist_name': 'The Casualties', 'track_name': 'Leaders of Today', 'energy': 1.0}, 'sort': [1.0]}, {'_index': 'spotify-insights', '_id': 'yhwOYJYBMkMmnc_ONDIj', '_score': None, '_source': {'artist_name': 'Nature Recordings', 'track_name': 'Static Rains', 'energy': 1.0}, 'sort': [1.0]}, {'_index': 'spotify-insights', '_id': 'yxwOYJYBMkMmnc_ONDIj', '_score': None, '_source': 

6-. Top 10 canciones más "acústicas".

In [ ]:
def showSongsWhitMostAcustic(self):
        try:
            response=self.connection.search(
                index=self.index,
                body={
                    "size": 0,
                    "aggs": {
                        "top_acoustic_songs": {
                        "top_hits": {
                            "size": 10,
                            "_source": ["track_name", "artist_name", "acousticness"],
                            "sort": [
                            {
                                "acousticness": {
                                "order": "desc"
                                }
                            }
                            ]
                        }
                        }
                    }
                    }
            )
            return response
            
        except Exception as e:
            print(f"Error: {str(e)}")
ElasticSearchProvider.showSongsWhitMostAcustic=showSongsWhitMostAcustic

In [23]:
provider = ElasticSearchProvider()
response = provider.showSongsWhitMostAcustic()
if response:
    print("Respuesta obtenida:")
    print(response)
else:
    print("No se obtuvo respuesta válida.")

AttributeError: 'ElasticSearchProvider' object has no attribute 'showSongsWhitMostAcustic'

7.- Promedio de "valence" por año.

In [ ]:
def showPromValencebyYear(self):
        try:
            response=self.connection.search(
                index=self.index,
                body={
                    "size": 0,
                    "aggs": {
                        "avg_valence_by_year": {
                        "terms": {
                            "field": "year",
                            "size": 10000  
                        },
                        "aggs": {
                            "avg_valence": {
                            "avg": {
                                "field": "valence"
                            }
                            }
                        }
                        }
                    }
                    }
            )
            return response
            
        except Exception as e:
            print(f"Error: {str(e)}")
            
ElasticSearchProvider.showPromValencebyYear=showPromValencebyYear

In [ ]:
provider = ElasticSearchProvider()
response = provider.showPromValencebyYear()
if response:
    print("Respuesta obtenida:")
    print(response)
else:
    print("No se obtuvo respuesta válida.")

8.- Distribución de las canciones por "mode" (mayoría mayor o menor).

In [ ]:
def showModebySongs(self):
        try:
            response=self.connection.search(
                index=self.index,
                body={
                    "size": 0,
                    "aggs": {
                        "mode_distribution": {
                        "terms": {
                            "field": "mode",
                            "size": 10
                        },
                        "aggs": {
                            "count_by_mode": {
                            "value_count": {
                                "field": "mode"
                            }
                            }
                        }
                        }
                    }
                    }
            )
            return response
            
        except Exception as e:
            print(f"Error: {str(e)}")

ElasticSearchProvider.showModebySongs=showModebySongs

In [ ]:
provider = ElasticSearchProvider()
response = provider.showModebySongs()
if response:
    print("Respuesta obtenida:")
    print(response)
else:
    print("No se obtuvo respuesta válida.")

9.- Promedio de duración de canciones por género.

In [ ]:
def showDurationBySongs(self):
        try:
            response=self.connection.search(
                index=self.index,
                body={
                    "size": 0,
                    "aggs": {
                        "genres": {
                        "terms": {
                            "field": "genre",  
                            "size": 20
                        },
                        "aggs": {
                            "avg_duration": {
                            "avg": {
                                "field": "duration_ms" 
                            }
                            }
                        }
                        }
                    }
                    }
            )
            return response
            
        except Exception as e:
            print(f"Error: {str(e)}")
            
ElasticSearchProvider.showDurationBySongs=showDurationBySongs

In [ ]:
provider = ElasticSearchProvider()
response = provider.showDurationBySongs()
if response:
    print("Respuesta obtenida:")
    print(response)
else:
    print("No se obtuvo respuesta válida.")

10.- Top 10 canciones más "loud".

In [ ]:
def showSongsWhitMostLoud(self):
        try:
            response=self.connection.search(
                index=self.index,
                body={
                    "size": 10,
                    "sort": [
                        {
                        "loudness": {
                            "order": "desc"  
                        }
                        }
                    ],
                    "_source": ["track_name", "artist_name", "loudness"]  
                    }
            )
            return response
            
        except Exception as e:
            print(f"Error: {str(e)}")
            
ElasticSearchProvider.showSongsWhitMostLoud=showSongsWhitMostLoud

In [ ]:
provider = ElasticSearchProvider()
response = provider.showSongsWhitMostLoud()
if response:
    print("Respuesta obtenida:")
    print(response)
else:
    print("No se obtuvo respuesta válida.")

11.-Top 10 canciones con mayor "speechiness".

In [ ]:
def showSongsWhitMostSpeech(self):
        try:
            response=self.connection.search(
                index=self.index,
                body={
                    "size": 10,
                    "sort": [
                        {
                        "speechiness": {
                            "order": "desc"  
                        }
                        }
                    ],
                    "_source": ["track_name", "artist_name", "speechiness"] 
                    }
            )
            return response
            
        except Exception as e:
            print(f"Error: {str(e)}")
            
ElasticSearchProvider.showSongsWhitMostSpeech=showSongsWhitMostSpeech

In [ ]:
provider = ElasticSearchProvider()
response = provider.showSongsWhitMostSpeech()
if response:
    print("Respuesta obtenida:")
    print(response)
else:
    print("No se obtuvo respuesta válida.")

12.-Distribución de los valores de "time_signature".

In [ ]:
def showTimeSignature(self):
        try:
            response=self.connection.search(
                index=self.index,
                body={
                    "size": 0,
                    "aggs": {
                        "time_signature_distribution": {
                        "terms": {
                            "field": "time_signature",
                            "size": 5  
                        }
                        }
                    }
                    }
            )
            return response
            
        except Exception as e:
            print(f"Error: {str(e)}")
            
ElasticSearchProvider.showTimeSignature=showTimeSignature

In [ ]:
provider = ElasticSearchProvider()
response = provider.showTimeSignature()
if response:
    print("Respuesta obtenida:")
    print(response)
else:
    print("No se obtuvo respuesta válida.")

13.-Top 10 canciones con mayor "instrumentalness".

In [ ]:
def showInstrumental(self):
        try:
            response=self.connection.search(
                index=self.index,
                body={
                    "size": 10,
                    "sort": [
                        {
                        "instrumentalness": {
                            "order": "desc"
                        }
                        }
                    ],
                    "_source": ["track_name", "artist_name", "instrumentalness"]
                    }
            )
            return response
            
        except Exception as e:
            print(f"Error: {str(e)}")

ElasticSearchProvider.showInstrumental=showInstrumental

In [ ]:
provider = ElasticSearchProvider()
response = provider.showInstrumental()
if response:
    print("Respuesta obtenida:")
    print(response)
else:
    print("No se obtuvo respuesta válida.")

14.- Las 10 canciones mas populares 

In [ ]:
def showMorePopularitySongs(self):
        try:
            response=self.connection.search(
                index=self.index,
                body={
                    "size": 10,
                    "sort": [
                        {
                        "popularity": {
                            "order": "desc"
                        }
                        }
                    ],
                    "_source": ["track_name", "artist_name", "popularity"]
                    }
            )
            return response
            
        except Exception as e:
            print(f"Error: {str(e)}")
            
ElasticSearchProvider.showMorePopularitySongs=showMorePopularitySongs

In [ ]:
provider = ElasticSearchProvider()
response = provider.showMorePopularitySongs()
if response:
    print("Respuesta obtenida:")
    print(response)
else:
    print("No se obtuvo respuesta válida.")

15.- las 10 canciones mas largas 

In [ ]:
def showMoreLongSongs(self):
        try:
            response=self.connection.search(
                index=self.index,
                body={
                    "size": 10,
                    "sort": [
                        {
                        "duration_ms": {
                            "order": "desc"
                        }
                        }
                    ],
                    "_source": ["track_name", "artist_name", "duration_ms"]
                    }
            )
            return response
            
        except Exception as e:
            print(f"Error: {str(e)}")

ElasticSearchProvider.showMoreLongSongs=showMoreLongSongs

In [ ]:
provider = ElasticSearchProvider()
response = provider.showMoreLongSongs()
if response:
    print("Respuesta obtenida:")
    print(response)
else:
    print("No se obtuvo respuesta válida.")